In [ ]:
import sqlite3
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

api_key = "c07c9d067d630faff2eed10b673199a3"

def get_film_info(title, year=None):
    print('fetching', title, year)
    #print number of current active threads
    print(threading.active_count())
    search_url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}"
    if year:
        search_url += f"&year={year}"
    search_response = requests.get(search_url)
    search_data = search_response.json()

    if search_data['results']:
        first_result = search_data['results'][0]
        film_id = first_result['id']

        # Fetch additional details (e.g., credits for director and actors)
        details_url = f"https://api.themoviedb.org/3/movie/{film_id}?api_key={api_key}&append_to_response=credits"
        details_response = requests.get(details_url)
        details_data = details_response.json()

        directors = [crew['name'] for crew in details_data['credits']['crew'] if crew['job'] == 'Director']
        actors = [actor['name'] for actor in details_data['credits']['cast'][:5]]  # Top 5 actors

        film_info = {
            'title': first_result['title'],
            'release_date': first_result.get('release_date', 'N/A'),
            'overview': first_result.get('overview', 'No overview available.'),
            'vote_average': first_result.get('vote_average', 'N/A'),
            'director': ', '.join(directors),
            'actors': ', '.join(actors),
            # 'box_office': details_data.get('revenue', 'N/A')  # Box office data if available
        }
    else:
        film_info = None

    return film_info

def update_database(film_info):
    conn = sqlite3.connect('my_letterboxd_data.db')
    cursor = conn.cursor()
    cursor.execute("""
    INSERT INTO film_details (movie_name, release_date, overview, director, actors)
    VALUES (?, ?, ?, ?, ?)
    ON CONFLICT(movie_name) DO UPDATE SET
    release_date=excluded.release_date,
    overview=excluded.overview,
    director=excluded.director,
    actors=excluded.actors
    """, (film_info['title'], film_info['release_date'], film_info['overview'], 
          film_info['director'], film_info['actors']))
    conn.commit()
    conn.close()

def main():
    conn = sqlite3.connect('my_letterboxd_data.db')
    cursor = conn.cursor()

    cursor.execute("SELECT DISTINCT movie_name, year FROM release_years")
    films = cursor.fetchall()

    #delete film_details table if it exists
    cursor.execute("DROP TABLE IF EXISTS film_details")

    #create film_details
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS film_details (
    movie_name TEXT PRIMARY KEY,
    release_date TEXT,
    overview TEXT,
    director TEXT,
    actors TEXT
    )
    """)


    conn.close()

    with ThreadPoolExecutor(max_workers=20) as executor:
        future_to_film = {executor.submit(get_film_info, film[0].strip('/'), film[1]): film for film in films}

        for future in tqdm(as_completed(future_to_film), total=len(films)):
            film_info = future.result()
            if film_info:
                update_database(film_info)

if __name__ == "__main__":
    main()


fetching the-flavor-of-green-tea-over-rice 1952
fetching the-fugitive-1993 1993


fetching the-fountain 2006
fetching the-french-connection 1971
fetching the-five-devils 2022
fetching the-florida-project 2017
fetching the-general 1926
fetching the-gift-2015-1 2015
fetching the-fugitive-kind 1960
fetching the-girl-who-leapt-through-time 2006
fetching the-fool-2014 2014
fetching the-girl-from-the-other-side 2019
fetching the-gleaners-and-i 2000
fetching the-front 1976
fetchingfetching the-gold-rush 1925
 the-girl-with-the-dragon-tattoo 2009
fetching the-good-lord-bird 2020
fetching the-good-boss 2021
fetching the-godfather-part-ii 1974
fetching the-godfather 1972
fetching the-good-the-bad-and-the-ugly 1966
fetching the-great-escape 1963
fetching the-graduate 1967
fetching the-great-dictator 1940
fetching the-hand-of-god 2021
fetching the-green-knight 2021
fetching the-green-mile 1999
fetching the-great-beauty 2013
fetching the-hate-u-give 2018
fetching the-handmaiden 2016
fetching the-great-white-silence 1922
fetching the-hateful-eight 2015
fetching the-great-gatsby-2

OperationalError: no such table: film_details

fetchingfetching the-last-picture-show 1971
fetching the-leopard 1963
 the-last-temptation-of-christ 1988


fetching the-life-aquatic-with-steve-zissou 2004
fetchingfetching the-last-emperor 1987
 the-lion-in-winter 1968
fetching the-life-of-oharu 1952
fetching the-line-the-cross-the-curve 1993
fetching the-lobster 2015
fetching the-little-drummer-girl-2018 2018
fetching the-lion-king 1994
fetching the-long-goodbye 1973
fetching the-loneliest-planet 2011
fetching the-lives-of-others 2006
fetching the-long-goodbye-2020 2020
fetching the-long-good-friday 1980
fetching the-look-of-love 2013
fetching the-machinist 2004
fetching the-man-who-sleeps 1974
fetching the-lunchbox 2013
fetching the-mad-womens-ball 2021
fetching the-man-who-shot-liberty-valance 1962
fetching the-lookout 2007
fetching the-making-of-the-girl-with-the-dragon-tattoo 2012
fetching the-man-from-nowhere 2010
fetching the-magnificent-seven 1960
fetching the-man-who-wasnt-there-2001 2001
fetching the-man-from-earth 2007
fetching the-maltese-falcon-1941 1941
fetching the-martian 2015
fetching the-master-2012 2012
fetching the-matr